In [5]:
%run ./3_DATA_PREPARATION.ipynb

In [7]:
# Prophet for All Stores
results = {}
results_metrics = []

for store in df.Store.unique():

    store_series = df.loc[df['Store'] == store]
    store_series = store_series[['Date', 'Weekly_Sales']]
    store_series.dropna(inplace=True)
    store_series.columns = ['Date', 'Sales']
    store_series = store_series[["Sales", "Date"]].resample("W",on ="Date").sum().reset_index()
    #store_df["Date"]=pd.to_datetime(store_df["Date"])
    #store_series.set_index("Date", inplace=True)

    store_series['Time'] = np.arange(len(store_series.index)) # add time step column

    store_series.rename(columns={'Date':'ds', 'Sales':'y'}, inplace=True)

    #https://medium.com/@Hong_Tang/timeseries-split-with-sklearn-tips-8162c83612b9

    # Time Series Split
    tscv = TimeSeriesSplit() # instantiate object from class. default parameter sets object to perform k=5 splits

    # Set date as index
    store_series.set_index('ds', inplace=True)
    store_series.sort_index(inplace=True)

    #prepare features dataframe (X) and target (y) as data series
    X = store_series.drop(labels=['y'], axis=1)
    y = store_series['y']

    # Split train test sets for both features and targets time-series class (tscv) class returns two arrays to mark train and test sets
    for train_index, test_index in tscv.split(X):
        X_train, X_test = X.iloc[train_index, :], X.iloc[test_index, :]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    # Summary of Train Test Split: number of records and date range for training and testing dataset.
    #print('The training dataset has', len(X_train), 'records, ranging from', pd.DataFrame(y_train).index.min(), 'to', pd.DataFrame(y_train).index.max())
    #print('The testing dataset has', len(X_test), 'records, ranging from', pd.DataFrame(y_test).index.min(), 'to', pd.DataFrame(y_test).index.max())

    #fit the model by instantiating a new Prophet object. Call fit method and pass in the train dataset
    m = Prophet()
    m.fit(pd.DataFrame(y_train).reset_index())

    """ 
    predictions are then made on a dataframe with a column ds containing the dates for which a prediction is to be made
    You can get a suitable dataframe that extends into the future a specified number of days using the helper
    method Prophet.make_future_dataframe. By default it will also include the dates from the history, so we will see the model fit as well.
    """
    future = m.make_future_dataframe(periods=23, freq='W') # predict next 23 weeeks
    #future.tail(40) uncomment no verify added periods

    """
    The predict method will assign each row in future a predicted value which it names yhat. If you pass in historical
    dates, it will provide an in-sample fit. The forecast object here is a new dataframe that includes a column yhat with the forecast,
    as well as columns for components and uncertainty intervals.
    """
    forecast = m.predict(future)
    #forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()

    """
    Model Performance
    """
    actual_sales = y_test.values
    predicted_sales = forecast.tail(23)['yhat'].values
    evaluate(actual_sales, predicted_sales)

    # model performance
    results_metrics.append(evaluate(actual_sales.values, predicted_sales))

    # save results
    results[store] = [X_train, X_test, y_train, actual_sales, predicted_sales]
    

    ## TODO: formato con pandas

INFO:prophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
c:\Users\mike_\anaconda3\lib\site-packages\prophet\forecaster.py:896: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)
c:\Users\mike_\anaconda3\lib\site-packages\prophet\forecaster.py:896: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)
c:\Users\mike_\anaconda3\lib\site-packages\prophet\forecaster.py:896: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)


TypeError: 'numpy.ndarray' object is not callable